In [ ]:
'''
1. parameter setting (save in local)
2. load dataset (optional : saving)
3. data preprocess (optional : saving)
4. modeling (optional: deliver to gcs OR DINHandler(default))
5. evaluation 
'''

In [2]:
import sys
sys.path.insert(0, 'work_dev/modeling/')
global_path = 'work_dev/modeling'

In [3]:
# load dataset
download_dataset_from_gcs = False
load_dataset_from_local = True
# execute data preprocess
save_preprocessed_data = False
load_preprocessed_data = True
# config, train_params
cfg_dump_path = f'{global_path}/show_u2i/config/din_standard.pickle'

# raw daataset
raw_dataset_path = f'{global_path}/show_u2i/dataset/raw_dataset_RUNTIME.pickle'

# preprocessed data  (save / load)
preprocessed_data_load_path = f'{global_path}/show_u2i/dataset/din_standard.pickle'
preprocessed_data_save_path = f'{global_path}/show_u2i/dataset/din_standard.pickle'

# run_time
run_time = '20230310'

In [4]:
# base_option
opt_dataset_blob_path = f'dataset/show/user2item/{run_time}'
opt_dataset = 'dataset.pickle'
opt_project_id = 'bf-data-prod-001'
opt_checkpoints_dir = 'checkpoints'
opt_experiment_name = 'test'
# general_config
COL2CATS_NAMES = 'col2label.pickle'
raw_dataset_path = raw_dataset_path.replace('RUNTIME', run_time)

In [4]:
# auto-argument
bucket_storage = f'machine-learning-models-{opt_project_id}'
base_path = f'{opt_checkpoints_dir}/{opt_experiment_name}'
if raw_dataset_path:
    opt_dataroot = '/'.join(raw_dataset_path.split('/')[:-1])
    raw_dataset_name = raw_dataset_path.split('/')[-1]
else:
    print('opt_dataroot not exist.')

In [33]:
# load config
import pickle
with open(cfg_dump_path, 'rb') as f:
    all_cfg = pickle.load(f)
config = all_cfg['config']
train_params = all_cfg['train_params']
train_params['dataroot'] = opt_dataroot

In [5]:
def data_preprocess_func(dataset, config, train_params):
    dataset = convert_columns_name(dataset, config)
    dataset = handle_missing_data(dataset, config)
    dataset = convert_data_type(dataset, config, mode='train')
    dataset = process_age(dataset, columns=['age'], age_range=20)
    dataset = process_category(dataset, config)
    dataset = aggregate_preference(dataset, config, mode='train')
    dataset, col2label2idx_new = encode_features(dataset, 
                                                 config, 
                                                 train_params, 
                                                 col2label2idx={}, 
                                                 mode='train')
    dataset = process_time_period(dataset, time_type=['min', 'hour'], postfix='_time_period')
    dataset = process_normalize_data(dataset, config)
    dataset = process_user_behavior_sequence(dataset, config, col2label2idx_new)
    return dataset, col2label2idx_new

In [6]:
from model_handler.din_handler import DINHandler

2023-03-31 14:46:46.993007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 14:46:48.316338: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-31 14:46:48.316478: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-31 14:46:48.316490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [7]:
from data_utils.processor import *
from data_utils.load import read_pickle, download_blob

In [8]:
'''
---LOAD DATASET AND META-DATA---
'''
file_to_download = [(opt_dataset, raw_dataset_name), (COL2CATS_NAMES, COL2CATS_NAMES)]

if download_dataset_from_gcs is True:
    for file, d_file in file_to_download:
        download_blob(bucket_name=bucket_storage,
                      source_blob_name=f'{opt_dataset_blob_path}/{base_path}/{file}',
                      destination_file_name=f'{opt_dataroot}/{d_file}')
if load_dataset_from_local is True:
    print('Starting load dataset from local.....')
    dataset = read_pickle(file_name='dataset.pickle', base_path=opt_dataroot)
    all_cats = read_pickle(file_name=COL2CATS_NAMES, base_path=opt_dataroot)

Starting load dataset from local.....


In [9]:
# show dataset
dataset.columns

Index(['userid', 'uuid', 'timestamp', 'date', 'hour', 'age', 'gender',
       'gamapay_used', 'title', 'final_score', '1_last_day_total_view_count',
       '1_last_day_total_click_count', '1_last_day_interaction_count',
       '3_last_day_total_view_count', '3_last_day_total_click_count',
       '5_last_day_total_view_count', '5_last_day_total_click_count',
       '5_last_day_interaction_count', 'user_category', 'user_title_embedding',
       'is_adult', 'publish_time', 'cat0', 'content_ner',
       'item_title_embedding', 'y', 'planet_news_used', 'planet_comic_used',
       'planet_novel_used', 'backpack_used', 'game_used', 'shopping_used',
       'gash_used', 'club_user_category', 'club_user_title_embedding'],
      dtype='object')

In [11]:
# save / load preprocessed data 
if load_preprocessed_data is False:
    dataset, col2label2idx_new = data_preprocess_func(dataset, config, train_params)
else:
    print('Starting load preprocessed data......')
    with open(preprocessed_data_load_path, 'rb') as f:
        prepreprocess_data = pickle.load(f)
    dataset = prepreprocess_data['dataset']
    col2label2idx_new = prepreprocess_data['col2label2idx_new']
if save_preprocessed_data and preprocessed_data_save_path:
    prepreprocess_data = {
        'dataset': dataset, 
        'col2label2idx_new': col2label2idx_new
    }
    with open(preprocessed_data_save_path, 'wb') as f:
        pickle.dump(prepreprocess_data, f)
else:
    print('dont save dataset or dataset_dump_path not exist.')

starting convert_columns_name...
starting handle_missing_data...


Pandas Apply:   0%|          | 0/265135 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/265135 [00:00<?, ?it/s]

starting convert_data_type...


Pandas Apply:   0%|          | 0/265135 [00:00<?, ?it/s]

...

---start aggregate_preference---


Pandas Apply:   0%|          | 0/265135 [00:00<?, ?it/s]

---start encode_features---
all_cats :  dict_keys(['cat0'])
---start process_time_period---
---start process_normalize_data---


Pandas Apply:   0%|          | 0/265135 [00:00<?, ?it/s]

---start process_user_behavior_sequence---


In [34]:
# evaluation
import random
import numpy as np

dat = dataset
train_rate = 0.8

data_size = dat.shape[0]
train_size = int(data_size * train_rate)
val_size = data_size - train_size
data_index = random.sample([i for i in range(data_size)], data_size)
train_index = data_index[:train_size]
val_index = data_index[train_size:]

In [35]:
train_y = dat[['y']].iloc[train_index]
val_y = dat[['y']].iloc[val_index]
train_x = dat.iloc[train_index]
val_x = dat.iloc[val_index]

In [36]:
# train
model_handler = DINHandler(col2label2idx=col2label2idx_new,
                           config=config,
                           mode='train',
                           **train_params)
model_handler.train(x_train=train_x, y_train=train_y, train_params=train_params)

cpu
Train on 212108 samples, validate on 0 samples, 829 steps per epoch


829it [00:11, 70.14it/s]


Epoch 1/10
11s - loss:  0.2549 - binary_crossentropy:  0.1732


829it [00:12, 65.27it/s]


Epoch 2/10
12s - loss:  0.0994 - binary_crossentropy:  0.0995


...

Epoch 8/10
11s - loss:  0.0879 - binary_crossentropy:  0.0879


829it [00:13, 63.03it/s]


Epoch 9/10
13s - loss:  0.0870 - binary_crossentropy:  0.0870


829it [00:11, 70.33it/s]

Epoch 10/10
11s - loss:  0.0863 - binary_crossentropy:  0.0863


In [37]:
val_y_pred = model_handler.predict(val_x).reshape(-1)

In [38]:
from eval_utils.get_metrics import get_overall_metrics, get_average_metrics
metric_type_list = ['precision', 'recall', 'f1', 'auc', 'roc_curve']
over_all_metrics = get_overall_metrics(val_y, val_y_pred, metric_type_list)
for metrics in metric_type_list[:-1]:
    score = over_all_metrics[metrics]
    if isinstance(score, float):
        print(f'{metrics} : ', score)
    else:
        print(f'{metrics} : ', score[-1])

precision :  0.7724329324699353
recall :  0.5533465871438038
f1 :  0.6447876447876448
auc :  0.9685319929357713


In [39]:
val_x['y_pred'] = val_y_pred
val_x_ = val_x.rename(columns={'y': 'y_true'})
for k in [5,10,15,20]:
    score = get_average_metrics(df=val_x_,
                         k=k,
                         metric_type='ndcg',
                         groupby_key='userid')
    print(f'NDCG@K={str(k)} : ', score)

/tmp/ipykernel_11228/2460681298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_x['y_pred'] = val_y_pred


NDCG@K=5 :  0.2637508888624586
NDCG@K=10 :  0.2658301301969786
NDCG@K=15 :  0.2665214737862511
NDCG@K=20 :  0.2666796518436062
